In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Using Vertex AI Matching Engine and Vertex AI Embeddings for Text for StackOverflow Questions 
![ ](https://www.google-analytics.com/collect?v=2&tid=G-L6X3ECH596&cid=1&en=page_view&sid=1&dt=sdk_matching_engine_create_stack_overflow_embeddings_vertex.ipynb&dl=notebooks%2Fofficial%2Fmatching_engine%2Fsdk_matching_engine_create_stack_overflow_embeddings_vertex.ipynb)
<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/matching_engine/sdk_matching_engine_create_stack_overflow_embeddings_vertex.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/matching_engine/sdk_matching_engine_create_stack_overflow_embeddings_vertex.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
      <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/matching_engine/sdk_matching_engine_create_stack_overflow_embeddings_vertex">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Overview

This example demonstrates how to encode text embeddings using the Vertex AI Embeddings for Text service and the StackOverflow dataset. These are uploaded to the Vertex AI Matching Engine service, which is a high-scale, low-latency solution, for finding similar vectors from a large corpus. Matching Engine is a fully managed offering, further reducing operational overhead. It is built upon [Approximate Nearest Neighbor (ANN) technology](https://ai.googleblog.com/2020/07/announcing-scann-efficient-vector.html) developed by Google Research.

Learn more about [Vertex AI Matching Engine](https://cloud.google.com/vertex-ai/docs/matching-engine/overview) and [Vertex AI Embeddings for Text](https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-text-embeddings).

### Objective

In this notebook, you learn how to encode text embeddings, create an Approximate Nearest Neighbor (ANN) index, and query against indexes.

This tutorial uses the following Google Cloud ML services:

- `Vertex AI Matching Engine`
- `Vertex AI Embeddings for Text`

The steps performed include:

* Convert a BigQuery dataset to embeddings
* Create an index
* Upload embeddings to the index
* Create an index endpoint
* Deploy the index to the index endpoint
* Perform an online query
* Add metadata to a Redis store

### Dataset

The dataset used for this tutorial is the [StackOverflow dataset](https://console.cloud.google.com/marketplace/product/stack-exchange/stack-overflow).

> Stack Overflow is the largest online community for programmers to learn, share their knowledge, and advance their careers. Updated on a quarterly basis, this BigQuery dataset includes an archive of Stack Overflow content, including posts, votes, tags, and badges. This dataset is updated to mirror the Stack Overflow content on the Internet Archive, and is also available through the Stack Exchange Data Explorer.

## Installation

Install the latest version of Cloud Storage, BigQuery, and the Vertex AI SDK for Python.

In [1]:
# Install the packages
! pip3 install --upgrade google-cloud-aiplatform \
                        google-cloud-storage \
                        'google-cloud-bigquery[pandas]' 

!pip install --user google-cloud-aiplatform==1.25.0


  Using cached google_cloud_aiplatform-1.26.1-py2.py3-none-any.whl (2.6 MB)
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.25.0
    Uninstalling google-cloud-aiplatform-1.25.0:
      Successfully uninstalled google-cloud-aiplatform-1.25.0
  Using cached google_cloud_aiplatform-1.25.0-py2.py3-none-any.whl (2.6 MB)
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


### Colab only: Uncomment the following cell to restart the kernel

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## Before you begin
#### Set your project ID

If you don't know your project ID, try the following:
* Run `gcloud config list`
* Run `gcloud projects list`
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
! gcloud config list

In [2]:
PROJECT_ID = "[YOUR-PROJECT-ID]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

Updated property [core/project].


#### Region

You can also change the `REGION` variable used by Vertex AI. To learn more about supported regions, see the [Vertex AI regions page](https://cloud.google.com/vertex-ai/docs/general/locations).

In [3]:
REGION = "us-central1"  # @param {type: "string"}

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**
* Do nothing as you are already authenticated.

**2. Local JupyterLab instance, uncomment and run:**

In [6]:
# ! gcloud auth login

**3. Colab, uncomment and run:**

In [7]:
# from google.colab import auth
# auth.authenticate_user()

**4. Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

* Authentication: Rerun the `gcloud auth login` command in the Vertex AI Workbench notebook terminal when you are logged out and need the credential again.

### Colab only: Uncomment the following cell to restart the kernel

In [8]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [13]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [4]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

## Prepare the data

You use the [Stack Overflow dataset](https://console.cloud.google.com/marketplace/product/stack-exchange/stack-overflow) of question and answers hosted on BigQuery.

> This public dataset is hosted in Google BigQuery and is included in BigQuery's 1TB/mo of free tier processing. This means that each user receives 1TB of free BigQuery processing every month, which can be used to run queries on this public dataset.

The BigQuery table is too large to fit into memory, so you need to write a generator called `query_bigquery_chunks` to yield chunks of the dataframe for processing. Additionally, an extra column `title_with_body` is added, which is a concatenation of the question title and body.

In [4]:
import math
from typing import Any, Generator

import pandas as pd
from google.cloud import bigquery

client = bigquery.Client(project=PROJECT_ID)
QUERY_TEMPLATE = """
        SELECT distinct q.id, q.title, q.body
        FROM (SELECT * FROM `bigquery-public-data.stackoverflow.posts_questions` where Score>0 ORDER BY View_Count desc) AS q 
        LIMIT {limit} OFFSET {offset};
        """


def query_bigquery_chunks(
    max_rows: int, rows_per_chunk: int, start_chunk: int = 0
) -> Generator[pd.DataFrame, Any, None]:
    for offset in range(start_chunk, max_rows, rows_per_chunk):
        query = QUERY_TEMPLATE.format(limit=rows_per_chunk, offset=offset)
        query_job = client.query(query)
        rows = query_job.result()
        df = rows.to_dataframe()
        df["title_with_body"] = df.title + "\n" + df.body
        yield df

In [5]:
# Get a dataframe of 1000 rows for demonstration purposes
df = next(query_bigquery_chunks(max_rows=1000, rows_per_chunk=1000))

# Examine the data
df.head()

,id,title,body,title_with_body
0,31300860,Manager doesn't work for Abstract Models,"<p>I am getting this error, however I'm not su...",Manager doesn't work for Abstract Models\n<p>I...
1,29580231,How to run a KeyListener at the same time than...,"<p>I'm actually coding pacman for a project, b...",How to run a KeyListener at the same time than...
2,29436716,"why do you need ""if instance is None"" in __get...",<p>I get the following example from Effective ...,"why do you need ""if instance is None"" in __get..."
3,29540931,Font for the interpretation line of a barcode,<p>How can I know which (default) font is used...,Font for the interpretation line of a barcode\...
4,29682863,Kinect 1.8 colorframe and depthframe not coord...,<p>My program has a problem with poor coordina...,Kinect 1.8 colorframe and depthframe not coord...


#### Instantiate the text encoding model

Use the [Vertex AI Embeddings for Text API](https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-text-embeddings) developed by Google for converting text to embeddings.

> Text embeddings are a dense vector representation of a piece of content such that, if two pieces of content are semantically similar, their respective embeddings are located near each other in the embedding vector space. This representation can be used to solve common NLP tasks, such as:
> - Semantic search: Search text ranked by semantic similarity.
> - Recommendation: Return items with text attributes similar to the given text.
> - Classification: Return the class of items whose text attributes are similar to the given text.
> - Clustering: Cluster items whose text attributes are similar to the given text.
> - Outlier Detection: Return items where text attributes are least related to the given text.

#### Defining an encoding function

Define a function to be used later that will take sentences and convert them to embeddings.

In [6]:
from typing import List, Optional

# Load the "Vertex AI Embeddings for Text" model
from vertexai.preview.language_models import TextEmbeddingModel

model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")


# Define an embedding method that uses the model
def encode_texts_to_embeddings(sentences: List[str]) -> List[Optional[List[float]]]:
    try:
        embeddings = model.get_embeddings(sentences)
        return [embedding.values for embedding in embeddings]
    except Exception:
        return [None for _ in range(len(sentences))]

#### Define two more helper functions for converting text to embeddings

- generate_batches: According to the documentation, each request can handle up to 5 text instances. Therefore, this method splits `sentences` into batches of 5 before sending to the embedding API.
- encode_text_to_embedding_batched: This method calls `generate_batches` to handle batching and then calls the embedding API via `encode_texts_to_embeddings`. It also handles rate-limiting using `time.sleep`. For production use cases, you would want a more sophisticated rate-limiting mechanism that takes retries into account.

In [7]:
import functools
import time
from concurrent.futures import ThreadPoolExecutor
from typing import Generator, List, Tuple

import numpy as np
from tqdm.auto import tqdm


# Generator function to yield batches of sentences
def generate_batches(
    sentences: List[str], batch_size: int
) -> Generator[List[str], None, None]:
    for i in range(0, len(sentences), batch_size):
        yield sentences[i : i + batch_size]


def encode_text_to_embedding_batched(
    sentences: List[str], api_calls_per_second: int = 10, batch_size: int = 5
) -> Tuple[List[bool], np.ndarray]:

    embeddings_list: List[List[float]] = []

    # Prepare the batches using a generator
    batches = generate_batches(sentences, batch_size)

    seconds_per_job = 1 / api_calls_per_second

    with ThreadPoolExecutor() as executor:
        futures = []
        for batch in tqdm(
            batches, total=math.ceil(len(sentences) / batch_size), position=0
        ):
            futures.append(
                executor.submit(functools.partial(encode_texts_to_embeddings), batch)
            )
            time.sleep(seconds_per_job)

        for future in futures:
            embeddings_list.extend(future.result())

    is_successful = [
        embedding is not None for sentence, embedding in zip(sentences, embeddings_list)
    ]
    embeddings_list_successful = np.squeeze(
        np.stack([embedding for embedding in embeddings_list if embedding is not None])
    )
    return is_successful, embeddings_list_successful

#### Test the encoding function

Encode a subset of data and see if the embeddings and distance metrics make sense.

In [8]:
# Encode a subset of questions for validation
questions = df.title.tolist()[:500]
is_successful, question_embeddings = encode_text_to_embedding_batched(
    sentences=df.title.tolist()[:500]
)

# Filter for successfully embedded sentences
questions = np.array(questions)[is_successful]

  0%|          | 0/100 [00:00<?, ?it/s]

Save the dimension size for later usage when creating the index.

In [9]:
DIMENSIONS = len(question_embeddings[0])

print(DIMENSIONS)

768


##### Sort questions in order of similarity.

According to the [embedding documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-text-embeddings#colab_example_of_semantic_search_using_embeddings), the similarity of embeddings is calculated using the dot-product.

- Calculate vector similarity using `np.dot`
- Sort by similarity
- Print results for inspection

In [10]:
import random

question_index = random.randint(0, 99)

print(f"Query question = {questions[question_index]}")

# Get similarity scores for each embedding by using dot-product.
scores = np.dot(question_embeddings[question_index], question_embeddings.T)

# Print top 20 matches
for index, (question, score) in enumerate(
    sorted(zip(questions, scores), key=lambda x: x[1], reverse=True)[:20]
):
    print(f"\t{index}: {question}: {score}")

Query question = How to set an .ajax() responseText to a variable
	0: How to set an .ajax() responseText to a variable: 0.999999251201694
	1: Modify django template variable in Jquery code: 0.702946933499988
	2: How to check if ajax call exists using jQuery?: 0.6498564447979907
	3: String to jQuery function: 0.6483313807663494
	4: How to call function defined using jquery in js?: 0.6366725051945362
	5: How do I assign a var into a Hash?: 0.6358129717377813
	6: how do i get this json data from php server in android: 0.6342202571320947
	7: Jquery Ajax JSONP error not firing: 0.6339935000742585
	8: How to respond to SyntaxError in javascript: 0.6262996012948951
	9: What is the JSON global object?: 0.6222843873156363
	10: Determine what jQuery script is called when a button is clicked: 0.6194055891290187
	11: angular 1 json pipe causing Lexer Error: Unterminated quote: 0.6166759112535305
	12: How to resolve Uncaught reference error when attempting to call a JavaScript function: 0.614658748

#### Save the embeddings in JSONL format

The data must be formatted in JSONL format, which means each embedding dictionary is written as an individual JSON object on its own line.

See more information in the docs at [Input data format and structure](https://cloud.google.com/vertex-ai/docs/matching-engine/match-eng-setup/format-structure#data-file-formats).

In [11]:
import tempfile
from pathlib import Path

# Create temporary file to write embeddings to
embeddings_file_path = Path(tempfile.mkdtemp())

print(f"Embeddings directory: {embeddings_file_path}")

Embeddings directory: /var/tmp/tmpytap78qj


Write embeddings in batches to prevent out-of-memory errors

In [12]:
import gc
import json

BQ_NUM_ROWS = 5000
BQ_CHUNK_SIZE = 1000
BQ_NUM_CHUNKS = math.ceil(BQ_NUM_ROWS / BQ_CHUNK_SIZE)

START_CHUNK = 0

# Create a rate limit of 300 requests per minute. Adjust this depending on your quota.
API_CALLS_PER_SECOND = 300 / 60
# According to the docs, each request can process 5 instances per request
ITEMS_PER_REQUEST = 5

# Loop through each generated dataframe, convert
for i, df in tqdm(
    enumerate(
        query_bigquery_chunks(
            max_rows=BQ_NUM_ROWS, rows_per_chunk=BQ_CHUNK_SIZE, start_chunk=START_CHUNK
        )
    ),
    total=BQ_NUM_CHUNKS - START_CHUNK,
    position=-1,
    desc="Chunk of rows from BigQuery",
):
    # Create a unique output file for each chunk
    chunk_path = embeddings_file_path.joinpath(
        f"{embeddings_file_path.stem}_{i+START_CHUNK}.json"
    )
    with open(chunk_path, "a") as f:
        id_chunk = df.id

        # Convert batch to embeddings
        is_successful, question_chunk_embeddings = encode_text_to_embedding_batched(
            sentences=df.title_with_body,
            api_calls_per_second=API_CALLS_PER_SECOND,
            batch_size=ITEMS_PER_REQUEST,
        )

        # Append to file
        embeddings_formatted = [
            json.dumps(
                {
                    "id": str(id),
                    "embedding": [str(value) for value in embedding],
                }
            )
            + "\n"
            for id, embedding in zip(id_chunk[is_successful], question_chunk_embeddings)
        ]
        f.writelines(embeddings_formatted)

        # Delete the DataFrame and any other large data structures
        del df
        gc.collect()

Chunk of rows from BigQuery:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

Upload the training data to a Google Cloud Storage bucket.

In [14]:
remote_folder = f"{BUCKET_URI}/{embeddings_file_path.stem}/"
! gsutil -m cp -r {embeddings_file_path}/* {remote_folder}

Copying file:///var/tmp/tmpytap78qj/tmpytap78qj_0.json [Content-Type=application/json]...
Copying file:///var/tmp/tmpytap78qj/tmpytap78qj_2.json [Content-Type=application/json]...
Copying file:///var/tmp/tmpytap78qj/tmpytap78qj_1.json [Content-Type=application/json]...
Copying file:///var/tmp/tmpytap78qj/tmpytap78qj_3.json [Content-Type=application/json]...
Copying file:///var/tmp/tmpytap78qj/tmpytap78qj_4.json [Content-Type=application/json]...
\ [5/5 files][ 88.5 MiB/ 88.5 MiB] 100% Done                                    
Operation completed over 5 objects/88.5 MiB.                                     


## Create Indexes


### Create ANN Index (for Production Usage)

In [15]:
DISPLAY_NAME = "stack_overflow"
DESCRIPTION = "question titles and bodies from stackoverflow"

Create the index configuration


In [16]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_URI)

In [17]:
DIMENSIONS = 768

tree_ah_index = aiplatform.MatchingEngineIndex.create_tree_ah_index(
    display_name=DISPLAY_NAME,
    contents_delta_uri=remote_folder,
    dimensions=DIMENSIONS,
    approximate_neighbors_count=150,
    distance_measure_type="DOT_PRODUCT_DISTANCE",
    leaf_node_embedding_count=500,
    leaf_nodes_to_search_percent=80,
    description=DESCRIPTION,
)

Creating MatchingEngineIndex
Create MatchingEngineIndex backing LRO: projects/186578989474/locations/us-central1/indexes/6254892951999086592/operations/10086026320019456
MatchingEngineIndex created. Resource name: projects/186578989474/locations/us-central1/indexes/6254892951999086592
To use this MatchingEngineIndex in another session:
index = aiplatform.MatchingEngineIndex('projects/186578989474/locations/us-central1/indexes/6254892951999086592')


In [18]:
INDEX_RESOURCE_NAME = tree_ah_index.resource_name
INDEX_RESOURCE_NAME

'projects/186578989474/locations/us-central1/indexes/6254892951999086592'

Using the resource name, you can retrieve an existing MatchingEngineIndex.

In [19]:
tree_ah_index = aiplatform.MatchingEngineIndex(index_name=INDEX_RESOURCE_NAME)

## Create an IndexEndpoint

In [20]:
my_index_endpoint = aiplatform.MatchingEngineIndexEndpoint.create(
    display_name=DISPLAY_NAME,
    description=DISPLAY_NAME,
    public_endpoint_enabled=True,
)

Creating MatchingEngineIndexEndpoint
Create MatchingEngineIndexEndpoint backing LRO: projects/186578989474/locations/us-central1/indexEndpoints/6875826750622793728/operations/7334064920331288576
MatchingEngineIndexEndpoint created. Resource name: projects/186578989474/locations/us-central1/indexEndpoints/6875826750622793728
To use this MatchingEngineIndexEndpoint in another session:
index_endpoint = aiplatform.MatchingEngineIndexEndpoint('projects/186578989474/locations/us-central1/indexEndpoints/6875826750622793728')


## Deploy Indexes

### Deploy ANN Index

In [21]:
DEPLOYED_INDEX_ID = "deployed_index_id_unique"

DEPLOYED_INDEX_ID

'deployed_index_id_uniquetest'

In [22]:
my_index_endpoint = my_index_endpoint.deploy_index(
    index=tree_ah_index, deployed_index_id=DEPLOYED_INDEX_ID
)

my_index_endpoint.deployed_indexes

Deploying index MatchingEngineIndexEndpoint index_endpoint: projects/186578989474/locations/us-central1/indexEndpoints/6875826750622793728
Deploy index MatchingEngineIndexEndpoint index_endpoint backing LRO: projects/186578989474/locations/us-central1/indexEndpoints/6875826750622793728/operations/9217695464478998528
MatchingEngineIndexEndpoint index_endpoint Deployed index. Resource name: projects/186578989474/locations/us-central1/indexEndpoints/6875826750622793728


[id: "deployed_index_id_uniquetest"
index: "projects/186578989474/locations/us-central1/indexes/6254892951999086592"
create_time {
  seconds: 1687792990
  nanos: 20034000
}
index_sync_time {
  seconds: 1687793836
  nanos: 782118000
}
automatic_resources {
  min_replica_count: 2
  max_replica_count: 2
}
deployment_group: "default"
]

#### Verify number of declared items matches the number of embeddings

Each IndexEndpoint can have multiple indexes deployed to it. For each index, you can retrieved the number of deployed vectors using the `index_endpoint._gca_resource.index_stats.vectors_count`. The numbers may not match exactly due to potential failures using the embedding service.

In [23]:
number_of_vectors = sum(
    aiplatform.MatchingEngineIndex(
        deployed_index.index
    )._gca_resource.index_stats.vectors_count
    for deployed_index in my_index_endpoint.deployed_indexes
)

print(f"Expected: {BQ_NUM_ROWS}, Actual: {number_of_vectors}")

Expected: 5000, Actual: 5000


## Create Online Queries

After you build your indexes, you may query against the deployed index to find nearest neighbors.

Note: For the DOT_PRODUCT_DISTANCE distance type, the "distance" property returned with each MatchNeighbor actually refers to the similarity.

In [30]:
test_embeddings = encode_texts_to_embeddings(sentences=["Install GPU for Tensorflow"])

In [28]:
# Test query
NUM_NEIGHBOURS = 10

response = my_index_endpoint.find_neighbors(
    deployed_index_id=DEPLOYED_INDEX_ID,
    queries=test_embeddings,
    num_neighbors=NUM_NEIGHBOURS,
)

response

[[MatchNeighbor(id='44523060', distance=0.7644188404083252),
  MatchNeighbor(id='64299011', distance=0.7430848479270935),
  MatchNeighbor(id='66432258', distance=0.7188020944595337),
  MatchNeighbor(id='70371687', distance=0.7165565490722656),
  MatchNeighbor(id='60374967', distance=0.7163087725639343),
  MatchNeighbor(id='68596830', distance=0.7154033184051514),
  MatchNeighbor(id='44490344', distance=0.6829164028167725),
  MatchNeighbor(id='64214847', distance=0.6821012496948242),
  MatchNeighbor(id='57005539', distance=0.67934650182724),
  MatchNeighbor(id='10937250', distance=0.6772756576538086)]]

Verify that the retrieved results are relevant by checking the StackOverflow links.

In [31]:
for match_index, neighbor in enumerate(response[0]):
    print(f"https://stackoverflow.com/questions/{neighbor.id}")

https://stackoverflow.com/questions/44523060
https://stackoverflow.com/questions/64299011
https://stackoverflow.com/questions/66432258
https://stackoverflow.com/questions/70371687
https://stackoverflow.com/questions/60374967
https://stackoverflow.com/questions/68596830
https://stackoverflow.com/questions/44490344
https://stackoverflow.com/questions/64214847
https://stackoverflow.com/questions/57005539
https://stackoverflow.com/questions/10937250


## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.
You can also manually delete resources that you created by running the following code.

In [ ]:
# Force undeployment of indexes and delete endpoint
my_index_endpoint.delete(force=True)

In [ ]:
# Delete indexes
tree_ah_index.delete()

In [ ]:
# Delete redis instance
! gcloud redis instances delete '{REDIS_INSTANCE_NAME}' --region {REGION} --quiet